In [1]:
import numpy as np 
import pandas as pd 
from pathlib import Path
from sklearn.metrics import log_loss
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
import random
import optuna
import time

import warnings
warnings.filterwarnings('ignore')

from dataclasses import dataclass
import os
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder,StandardScaler, QuantileTransformer, RobustScaler, PowerTransformer
from sklearn.feature_selection import VarianceThreshold



import seaborn as sns

import tensorflow as tf
from tensorflow.keras import Sequential, layers, Input, Model
from tensorflow.experimental import numpy as tfnp
from tensorflow.keras.losses import Loss
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline



from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import RobustScaler
from keras.optimizers import Adam
from keras import regularizers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pathlib import Path
from keras.layers import Input, Dense, concatenate
import tensorflow.keras.backend as K
from sklearn.utils import resample
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from keras.models import load_model

from sklearn.feature_selection import RFE
from sklearn.svm import SVC
import time
import copy

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
@dataclass
class DataHolder():
    folds : [pd.DataFrame]
        
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
def balanced_log_loss(y_true, y_pred,return_all=False):
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)

    p_1 = np.clip(y_pred, 1e-15, 1 - (1e-15))
    p_0 = 1 - p_1

    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1

    if return_all:
        return (log_loss_0 + log_loss_1) / 2, log_loss_0,log_loss_1
    return (log_loss_0 + log_loss_1) / 2
def preprocess_df(input_df: pd.DataFrame)->pd.DataFrame:
    input_df = input_df.rename(columns={'BD ': 'BD', 'CD ': 'CD', 'CW ': 'CW', 'FD ': 'FD'}) #?
    output_df = input_df.copy()
    str2int_dict = {}
    str2int_dict['EJ'] = {'A': 1.0, 'B': 0.0}
    for col in str2int_dict.keys():
            output_df[col] = output_df[col].map(str2int_dict[col])
    return output_df
    
def variance_threshold_selector(data, threshold=0.8):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [3]:
def split_on_column(column,train_df,greeks_df):
    train_group = [train_df[train_df['Id'].isin(greeks_df[greeks_df[column] == letter]['Id'])] for letter in greeks_df[column].unique()]

    grouped_splits = [np.array_split(grouped,CFG.n_folds) for grouped in train_group]

    data_holder_split = DataHolder( [pd.concat([split[i] for split in grouped_splits]) for i in range(CFG.n_folds)]
                                   )
    return data_holder_split

In [4]:
def xgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred)

def calc_log_loss_weight(y_true):
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1


def xgboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, passed_params,num_estimators):
    
    train_w0, train_w1 = calc_log_loss_weight(y_train)
    valid_w0, valid_w1 = calc_log_loss_weight(y_valid)
    xgb_train = xgb.DMatrix(data=x_train, label=y_train, weight=y_train.map({0: train_w0, 1: train_w1}))
    xgb_valid = xgb.DMatrix(data=x_valid, label=y_valid, weight=y_valid.map({0: valid_w0, 1: valid_w1}))
    
    model = xgb.train(
                passed_params, 
                dtrain = xgb_train, 
                num_boost_round = num_estimators, 
                evals = [(xgb_valid, 'eval')], 
                verbose_eval = False,
                custom_metric = xgb_metric, 
            )
    return model


In [5]:
def create_prose_model(n_inputs,optimizer,layer_size):
    inp = tf.keras.layers.Input(shape = (n_inputs, ))
    x0 = tf.keras.layers.BatchNormalization()(inp)
    
#     encoder = tf.keras.layers.GaussianNoise(.1)(x0)
    encoder = tf.keras.layers.Dense(layer_size)(x0)
    encoder = tf.keras.layers.BatchNormalization()(encoder)
    encoder = tf.keras.layers.Activation('swish')(encoder)

#     decoder = tf.keras.layers.Dropout(.1)(encoder)
    decoder = tf.keras.layers.Dense(n_inputs, name = 'decoder')(encoder)

    # Combine the outputs
    
    model = Model(inputs=inp, outputs=[decoder,encoder])
    loss = {'decoder': tf.keras.losses.MeanSquaredError()},
    model.compile(optimizer=optimizer, loss=loss)
    
    return model

from keras.layers import Input, Dense, LeakyReLU 
    
def create_model_github(n_inputs,optimizer,layer_size):
    #https://github.com/xxl4tomxu98/autoencoder-feature-extraction/tree/main
    visible = Input(shape=(n_inputs,))
    # encoder level 1
    e = Dense(n_inputs*2)(visible)
    e = tf.keras.layers.BatchNormalization()(e)
    e = LeakyReLU()(e)
    # encoder level 2
    e = Dense(n_inputs)(e)
    e = tf.keras.layers.BatchNormalization()(e)
    e = LeakyReLU()(e)
    # bottleneck
    #n_bottleneck = n_inputs # no compression of bottleneck
    n_bottleneck = round(float(n_inputs) / 2.0) # reduced nodes to 50
    bottleneck = Dense(n_bottleneck)(e)
    
    d = Dense(n_inputs)(bottleneck)
    d = tf.keras.layers.BatchNormalization()(d)
    d = LeakyReLU()(d)
    # decoder level 2
    d = Dense(n_inputs*2)(bottleneck)
    d = tf.keras.layers.BatchNormalization()(d)
    d = LeakyReLU()(d)
    # output layer
    output = Dense(n_inputs, activation='linear')(d)
    # define autoencoder model
    model = Model(inputs=visible, outputs=output)
    loss = {'decoder': tf.keras.losses.MeanSquaredError()},
    model.compile(optimizer=optimizer, loss=loss)
    
    return model


# Config details

In [6]:
class CFG:
    DATA_PATH = Path('/kaggle/input/icr-identify-age-related-conditions')
#     OOF_DATA_PATH = Path('./oof')
#     MODEL_DATA_PATH = Path('./models')
#     METHOD_LIST = ['lightgbm', 'xgboost', 'catboost']
    seed = 42 #52
    n_folds = 10 #replaced 20
    target_col = 'Class'
    metric = 'balanced_log_loss'
 
    

In [7]:
# from datetime import datetime
# times = greeks_df.Epsilon.copy()
# times[greeks_df.Epsilon != 'Unknown'] = greeks_df.Epsilon[greeks_df.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
# times[greeks_df.Epsilon == 'Unknown'] = np.nan
# train_df['times'] = times.fillna(np.nan).astype(float)

# Initialize features

In [8]:
def build_neutralizer(df,features,proportion):
    """  
    Builds neutralzied features, then trains a linear model to predict neutralized features from original
    features and return the coeffs of that model.
    """
    df['bias'] = 1.0
    neutralizer = {}
    neutralized_features = np.zeros((df.shape[0], len(features)))
    target = df[['Class', 'bias']].values
    for i, f in enumerate(features):
        # obtain corrected feature
        feature = df[f].values.reshape(-1, 1)
        coeffs = np.linalg.lstsq(target, feature)[0]
        neutralized_features[:, i] = (feature - (proportion * target.dot(coeffs))).squeeze()

    # train model to predict corrected features
    neutralizer = np.linalg.lstsq(df[features+['bias']].values, neutralized_features)[0]

    return neutralizer

def neutralize_array(array, neutralizer):
    neutralized_array = array.dot(neutralizer)
    return neutralized_array


# HP Tune start

In [9]:
# Set the maximum running time (in seconds) to 11h hours and 40 minutes because the seasion of kaggle in 12h using CPU
# max_running_time = 45 * 60 + 11 * 60 * 60 
max_running_time = 11.5 * 60 * 60 
# Start the timer
start_time = time.time()
GITHUB = False
def run(trial):
    
    final_valid_predictions = {}
    bs = []
    
    learning_rate    = trial.suggest_float("learning_rate", .01, 0.08)
    num_estimators   = trial.suggest_int("num_estimators", 50, 400)
    max_depth        = trial.suggest_int("max_depth", 2,5)
    bagging_fraction = trial.suggest_uniform("subsample", 0.3, .7)
    min_child_weight = trial.suggest_int("min_child_weight",10,25)
    lambda_          = trial.suggest_int('lambda_', 1, 10)
    scale            = trial.suggest_uniform("scale",1.5,6)
    

    
    
    
    
    
    quantiles        = trial.suggest_categorical("quantiles",[50,75,100])
    
    batch_size       = trial.suggest_categorical("batch_size",[16,32,64])
    ae_lr            = trial.suggest_float("ae_lr",.001,.01)
    epochs_          = trial.suggest_int("epochs_",50,140)
    if not GITHUB:
        layer_size       = trial.suggest_int("layer_size",10,45)
    
#     total_features   = trial.suggest_int("total_features",25,75)
# #     total_features = 50 + layer_size
#     if total_features < 30: 
#         min_feat_prop = .35
#     elif total_features < 40:
#         min_feat_prop = .3
#     elif  total_features < 45:
#         min_feat_prop = .25
#     else:
#         min_feat_prop = .2
        
    feature_fraction = trial.suggest_uniform("colsample_bytree", .1, .7)
    
    
    
    frames = []
    overall_mean = -1
    for seed in [56,720]:
        CFG.seed = seed
        seed_everything(CFG.seed)
        np.random.seed(CFG.seed)
        tf.random.set_seed(CFG.seed)

        failure_ind = False
#################################################################
# data handling    
#################################################################

        train_df = preprocess_df(pd.read_csv(CFG.DATA_PATH / 'train.csv'))
        greeks_df = pd.read_csv(CFG.DATA_PATH / 'greeks.csv')
        numerical_features = ['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 
                              'BD', 'BN', 'BP', 'BQ', 'BR', 'BZ',
                              'CB', 'CC', 'CD', 'CF', 'CH', 'CL', 
                              'CR', 'CS', 'CU', 'CW',
                              'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
                              'EB', 'EE', 'EG', 'EH', 'EL', 'EP', 'EU',
                              'FC', 'FD', 'FE', 'FI', 'FL', 'FR', 'FS',
                              'GB', 'GE', 'GF', 'GH', 'GI', 'GL']
        features = numerical_features

        ae_train_df = preprocess_df(pd.read_csv(CFG.DATA_PATH / 'train.csv'))
        ae_train_df = ae_train_df.fillna(ae_train_df.median())
        feature_list = list(set(ae_train_df.columns) - set(['Id','Class']))

    
    
        skewed_vars = features
        replace_vars = {}
        quantile_transformers = {}
        for skewed_var in skewed_vars:
            qt = QuantileTransformer(n_quantiles=quantiles,random_state=CFG.seed, output_distribution='normal')
            train_df[skewed_var + "_trans"] = qt.fit_transform(train_df[skewed_var].to_numpy().reshape(-1,1))
            replace_vars[skewed_var] = skewed_var + "_trans"
            quantile_transformers[skewed_var] = qt

        features = [replace_vars.get(feature,feature)  for feature in features]
        

        train_df = greeks_df.merge(train_df,on='Id')

        positives = train_df[train_df[CFG.target_col] == 1]
        negatives = train_df[train_df[CFG.target_col] == 0]

        positives_splits = np.array_split(positives, CFG.n_folds)
        negatives_splits = np.array_split(negatives, CFG.n_folds)

        data_holder_split_class = DataHolder( [pd.concat([positives_splits[i],negatives_splits[i]]) for i in range(CFG.n_folds)]
                                       )

        data_holder_split_alpha = split_on_column("Alpha",train_df,greeks_df)
        data_holder_split_gamma = split_on_column("Gamma",train_df,greeks_df)

        cv_holders = [data_holder_split_class,data_holder_split_alpha,data_holder_split_gamma]


    #################################################################
    ## run CV    
    #################################################################

        cv_strat_instances = []

        for cv_strat in tqdm(cv_holders):
            fold_instances = []
            scores = []
            for idx,fold_number in enumerate(range(CFG.n_folds)):
                test_idx = idx + 1 
                if idx == CFG.n_folds -1:
                    test_idx = 0
                temp_data = cv_strat.folds.copy()
                test = temp_data[test_idx].copy()
                valid = temp_data[idx].copy()
                if test_idx < idx:
                    del temp_data[idx]
                    del temp_data[test_idx]
                else:
                    del temp_data[test_idx]
                    del temp_data[idx]
                train = pd.concat(temp_data)


                ################################################################################################################################################
                # neutralize 
                ################################################################################################################################################

                proportion = .25
                ae_train = copy.deepcopy(ae_train_df.loc[train.index])
                ae_valid = copy.deepcopy(ae_train_df.loc[valid.index])
                ae_test = copy.deepcopy(ae_train_df.loc[test.index])
                
                ae_neut_temp = pd.concat([ae_train,ae_valid])
                
                neutralizer = build_neutralizer(ae_neut_temp, feature_list, proportion)
                ae_neut_temp[feature_list] = neutralize_array(ae_neut_temp[feature_list+['bias']].values, neutralizer)
                
                rows = []
                for idx,row in ae_test.iterrows():
                    rows.append(neutralize_array(np.asarray(row[feature_list].tolist() + [1.0]),neutralizer))
                ae_test[feature_list] = rows
                
                ae_train = ae_neut_temp.loc[train.index]
                ae_valid = ae_neut_temp.loc[valid.index]

                
                proportion = .25
                temp_train = pd.concat([copy.deepcopy(train),copy.deepcopy(valid)])
                temp_train = temp_train.fillna(temp_train.median())
                test = test.fillna(temp_train.median())
                
                neutralizer = build_neutralizer(temp_train, features, proportion)
                temp_train[features] = neutralize_array(temp_train[features+['bias']].values, neutralizer)
                
                rows = []
                for idx,row in test.iterrows():
                    rows.append(neutralize_array(np.asarray(row[features].tolist() + [1.0]),neutralizer))
                test[features] = rows
                
                train = temp_train.loc[train.index]
                valid = temp_train.loc[valid.index]
                    

                
                ################################################################################################################################################
                # VAE 
                ################################################################################################################################################

                
                ae_train = ae_train_df.loc[train.index]
                ae_valid = ae_train_df.loc[valid.index]
                ae_test = ae_train_df.loc[test.index]
                

                optimizer = Adam(learning_rate=ae_lr)
                BATCH_SIZE = batch_size
                
                
                if GITHUB:
                    layer_size = -1
                    model = create_model_github(len(feature_list),optimizer,layer_size)
                else:
                    model = create_prose_model(len(feature_list),optimizer,layer_size)


                train_ds = tf.data.Dataset.from_tensor_slices(  (ae_train[feature_list].to_numpy(),  (ae_train[feature_list].to_numpy()) ) ).batch(BATCH_SIZE)
                valid_ds = tf.data.Dataset.from_tensor_slices(  (ae_valid[feature_list].to_numpy(),  (ae_valid[feature_list].to_numpy()) ) ).batch(BATCH_SIZE)
                test_x_ds = tf.data.Dataset.from_tensor_slices(ae_test[feature_list].to_numpy()).batch(BATCH_SIZE)


                # Train the main model using the selected model's weights
                model.fit(train_ds,
                          validation_data=(valid_ds),
                          epochs=epochs_,
        #                   callbacks=[early_stopping],
                          verbose=0
                         )

                if GITHUB:
                    predicted_features_train= model.predict(tf.data.Dataset.from_tensor_slices(  (ae_train[feature_list].to_numpy() ) ).batch(BATCH_SIZE))
                    predicted_features_valid= model.predict(tf.data.Dataset.from_tensor_slices(  (ae_valid[feature_list].to_numpy() ) ).batch(BATCH_SIZE))
                    predicted_features_test = model.predict(test_x_ds)
                else:
                    predicted_features_train= model.predict(tf.data.Dataset.from_tensor_slices(  (ae_train[feature_list].to_numpy() ) ).batch(BATCH_SIZE))[1]
                    predicted_features_valid= model.predict(tf.data.Dataset.from_tensor_slices(  (ae_valid[feature_list].to_numpy() ) ).batch(BATCH_SIZE))[1]
                    predicted_features_test = model.predict(test_x_ds)[1]


#                 keep_columns = ['AB_trans', 'AF_trans', 'AM_trans', 'AX_trans', 'AY_trans', 'BD_trans',
#                                'BP_trans', 'BQ_trans', 'BR_trans', 'CC_trans', 'CD_trans', 'CF_trans',
#                                'CH_trans', 'CR_trans', 'DA_trans', 'DE_trans', 'DH_trans', 'DI_trans',
#                                'DL_trans', 'DN_trans', 'DU_trans', 'DV_trans', 'DY_trans', 'EB_trans',
#                                'EE_trans', 'EG_trans', 'EH_trans', 'EL_trans', 'FC_trans', 'FD_trans',
#                                'FI_trans', 'FL_trans', 'GE_trans', 'GH_trans', 'GL_trans']# top 35 arbitrary from svc of just base features
                x_train = pd.concat([train[features],pd.DataFrame(predicted_features_train,index=train.index,columns=[f'f{x}' for x in range(layer_size)])],axis=1)
                x_valid = pd.concat([valid[features],pd.DataFrame(predicted_features_valid,index=valid.index,columns=[f'f{x}' for x in range(layer_size)])],axis=1)
                x_test = pd.concat([test[features],pd.DataFrame(predicted_features_test,index=test.index,columns=[f'f{x}' for x in range(layer_size)])],axis=1)
                
                y_train = train['Class']
                y_valid = valid['Class']
                y_test = test['Class']

            ################################################################################################################################################
        
#                 x_train_filled_na = pd.concat([train[features],pd.DataFrame(predicted_features_train,index=train.index)],axis=1)
#                 x_train_filled_na = x_train_filled_na.fillna(x_train_filled_na.median())
            
                
                
#                 estimator = LogisticRegression()
#                 selector = RFE(estimator, n_features_to_select=total_features, step=1)
#                 selector = selector.fit(x_train_filled_na.to_numpy(), y_train.to_numpy())
                
                
#                 keep_columns = x_train.columns[selector.support_]
#                 x_train = x_train[keep_columns]
#                 x_valid = x_valid[keep_columns]
#                 x_test = x_test[keep_columns]
            
            ################################################################################################################################################
                xgb_params = {
                'objective': 'binary:logistic',
                'eval_metric': 'logloss',
                'learning_rate': learning_rate, 
                'max_depth': max_depth,
                'colsample_bytree': feature_fraction,
                'subsample': bagging_fraction,
                'lambda': lambda_,
                'min_child_weight': min_child_weight,
                'seed': CFG.seed,
                'scale_pos_weight': scale, 

            }

                model = xgboost_training(x_train, y_train, x_valid, y_valid,xgb_params,num_estimators)

                y_pred = model.predict(xgb.DMatrix(x_test))


                score,log_loss_0,log_loss_1 = balanced_log_loss(y_test, y_pred,return_all=True)
                fold_instances.append([score,log_loss_0,log_loss_1] + [float(x) for x in model.attributes().values()]) # best_iteration, best_ntree_limit, best_validation_score
                scores.append(score)
                
            cv_strat_instances.append(fold_instances)
            if np.mean(scores) > .24:
                failure_ind = True
                return np.mean(scores) + .12

        cv_strat_instances = np.asarray(cv_strat_instances)
        frame_scores = pd.DataFrame(cv_strat_instances[:,:,0],columns=[f"fold_{i}" for i in range(CFG.n_folds)],index=['Class','Alpha','Gamma'])
        log_loss_mean_details = pd.DataFrame(np.mean(cv_strat_instances[:,:,1:3].astype(float),axis=1),columns=["mean_log_loss0","mean_log_loss1" ],index=['Class','Alpha','Gamma'])
        log_loss_std_details = pd.DataFrame(np.std(cv_strat_instances[:,:,1:3].astype(float),axis=1),columns=["std_log_loss0","std_log_loss1" ],index=['Class','Alpha','Gamma'])
        overall_scores = frame_scores.agg([np.mean,np.std],axis=1).join(log_loss_mean_details).join(log_loss_std_details)[['mean','std','mean_log_loss0','std_log_loss0','mean_log_loss1','std_log_loss1']].mean()
        frames.append(overall_scores)
        
#     print(f"Balance Log loss:                        {overall_scores['mean']}")
#     print(f"std of Log loss across CV style:         {overall_scores['std']}")
#     print(f"Log loss class 0:                        {overall_scores['mean_log_loss0']}")
#     print(f"std of Log loss Class 0 across CV style: {overall_scores['std_log_loss0']}")
#     print(f"Log loss class 1:                        {overall_scores['mean_log_loss1']}")
#     print(f"std of Log loss Class 1 across CV style: {overall_scores['std_log_loss1']}")
#     print(f"Total tree sizes:                        {np.ravel(cv_strat_instances[:,:,4])}")

    
    overall_mean = np.mean([frame['mean'] for frame in frames])
    print(f"Balance Log loss:                        {overall_mean}")
    print(f"std of Log loss across CV style:         {np.mean([frame['std'] for frame in frames])}")
    print(f"Log loss class 0:                        {np.mean([frame['mean_log_loss0'] for frame in frames])}")
    print(f"std of Log loss Class 0 across CV style: {np.mean([frame['std_log_loss0'] for frame in frames])}")
    print(f"Log loss class 1:                        {np.mean([frame['mean_log_loss1'] for frame in frames])}")
    print(f"std of Log loss Class 1 across CV style: {np.mean([frame['std_log_loss1'] for frame in frames])}")

    
    return overall_mean


In [10]:
%%time
study = optuna.create_study(direction="minimize")
while True:
    # Check if the maximum running time has been exceeded
    elapsed_time = time.time() - start_time
    if elapsed_time > max_running_time:
        break
    
    # Optimize a single trial
    study.optimize(run, n_trials=1)
    
    # Print intermediate results
    print('Best trial:', study.best_trial.params)
    print('Current elapsed time:', elapsed_time)
    
# Get the best hyperparameters
best_params = study.best_trial.params
print('Best hyperparameters:', best_params)


[I 2023-07-24 17:07:59,448] A new study created in memory with name: no-name-79457eb7-bac7-4f45-957e-cf7941e46f95
  0%|          | 0/3 [00:00<?, ?it/s]

2/2 [==============================] - 0s 4ms/step


  0%|          | 0/3 [01:38<?, ?it/s]
[I 2023-07-24 17:09:37,835] Trial 0 finished with value: 0.47636407868101954 and parameters: {'learning_rate': 0.02648238749698697, 'num_estimators': 391, 'max_depth': 3, 'subsample': 0.3243929312024955, 'min_child_weight': 12, 'lambda_': 4, 'scale': 2.8241181629741794, 'quantiles': 75, 'batch_size': 32, 'ae_lr': 0.00909569173881386, 'epochs_': 121, 'layer_size': 38, 'colsample_bytree': 0.4926631260928611}. Best is trial 0 with value: 0.47636407868101954.


Best trial: {'learning_rate': 0.02648238749698697, 'num_estimators': 391, 'max_depth': 3, 'subsample': 0.3243929312024955, 'min_child_weight': 12, 'lambda_': 4, 'scale': 2.8241181629741794, 'quantiles': 75, 'batch_size': 32, 'ae_lr': 0.00909569173881386, 'epochs_': 121, 'layer_size': 38, 'colsample_bytree': 0.4926631260928611}
Current elapsed time: 0.03040766716003418


  0%|          | 0/3 [00:00<?, ?it/s]

...

Best trial: {'learning_rate': 0.05589995764758967, 'num_estimators': 321, 'max_depth': 5, 'subsample': 0.3765453750749054, 'min_child_weight': 18, 'lambda_': 4, 'scale': 2.026146030337454, 'quantiles': 75, 'batch_size': 32, 'ae_lr': 0.008218829335618042, 'epochs_': 55, 'layer_size': 13, 'colsample_bytree': 0.11157197454650197}
Current elapsed time: 41327.53030657768


  0%|          | 0/3 [00:00<?, ?it/s]

4/4 [==============================] - 0s 2ms/step


  0%|          | 0/3 [01:34<?, ?it/s]
[I 2023-07-25 04:39:24,690] Trial 620 finished with value: 0.4439767916961548 and parameters: {'learning_rate': 0.05499718689325983, 'num_estimators': 183, 'max_depth': 5, 'subsample': 0.36247381637537496, 'min_child_weight': 18, 'lambda_': 5, 'scale': 1.9164784240568409, 'quantiles': 75, 'batch_size': 16, 'ae_lr': 0.0033615005543627457, 'epochs_': 53, 'layer_size': 13, 'colsample_bytree': 0.13715661359627337}. Best is trial 562 with value: 0.4321325600620296.


Best trial: {'learning_rate': 0.05589995764758967, 'num_estimators': 321, 'max_depth': 5, 'subsample': 0.3765453750749054, 'min_child_weight': 18, 'lambda_': 4, 'scale': 2.026146030337454, 'quantiles': 75, 'batch_size': 32, 'ae_lr': 0.008218829335618042, 'epochs_': 55, 'layer_size': 13, 'colsample_bytree': 0.11157197454650197}
Current elapsed time: 41389.76427817345
Best hyperparameters: {'learning_rate': 0.05589995764758967, 'num_estimators': 321, 'max_depth': 5, 'subsample': 0.3765453750749054, 'min_child_weight': 18, 'lambda_': 4, 'scale': 2.026146030337454, 'quantiles': 75, 'batch_size': 32, 'ae_lr': 0.008218829335618042, 'epochs_': 55, 'layer_size': 13, 'colsample_bytree': 0.11157197454650197}
CPU times: user 16h 14min 42s, sys: 1h 25min 54s, total: 17h 40min 36s
Wall time: 11h 31min 25s


In [11]:
sorted([x.values[0] for x in study.trials])

[0.4321325600620296,
 0.4332277299642676,
 0.4352138391371531,
 0.4359543991211433,
 0.4359648691566923,
 0.43597972944522123,
 0.43602588431484857,
 0.43636639705358427,
 0.43683806441285705,
 0.43740668662334836,
 0.4374877821121551,
 0.437905847861815,
 0.4380732895858808,
 0.4381532379737034,
 0.4382617188801212,
 0.4385666082151095,
 0.4386107702375839,
 0.4386121151113132,
 0.4387689383495068,
 0.438841221254138,
 0.43889869813983445,
 0.439235759328566,
 0.4393039623070276,
 0.43952520908416476,
 0.43959281657329047,
 0.4396302998084542,
 0.4397011103600599,
 0.4397136101933269,
 0.43974946664351433,
 0.43976649575779764,
 0.439819132795481,
 0.4400421243246625,
 0.44021204549372994,
 0.44029889797097255,
 0.4403750713393723,
 0.44038912088202176,
 0.44053646626857024,
 0.44067997995841873,
 0.4407580212904389,
 0.4409819855675011,
 0.4410050744494961,
 0.4411037846197725,
 0.4412151966910285,
 0.4413061454044207,
 0.44134469791052366,
 0.4413605010269671,
 0.44179349202947577,
